In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.offline import iplot


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/iowa-liquor-sales/Iowa_Liquor_Sales.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
categorical = []
for col, value in df.iteritems():
    if value.dtypes == 'object':
        categorical.append(col)
        
numerical = df.columns.difference(categorical)

In [ ]:
df[numerical].isnull().sum()

In [ ]:
dfx = df[numerical]
dfx.head()

In [ ]:
df_summary = df.describe()

In [ ]:
def draw_axvlines(plt, col):
    mean = df_summary.loc["mean", col]
    q1 = df_summary.loc["25%", col]
    q2 = df_summary.loc["50%", col]
    q3 = df_summary.loc["75%", col]
    plt.axvline(mean, color = "g");              # Plotting a line to mark the mean 
    plt.axvline(q1, color = "b");                # Plotting a line to mark Q1 
    plt.axvline(q2, color = "navy");             # Plotting a line to mark Q2 
    plt.axvline(q3, color = "purple");           # Plotting a line to mark Q3
    plt.legend({"Mean": mean, "25%" : q1, "50%" : q2, "75%" : q3});


In [ ]:
fig, axes = plt.subplots(3, 2, figsize = (20,12));
fig.suptitle('Distribution');

sns.boxplot(dfx["Volume Sold (Gallons)"], ax = axes[0][0], color = "mediumslateblue");
axes[0][0].set(xlabel = 'Volume Sold Gallons');

pp = sns.distplot(df["Volume Sold (Gallons)"], ax = axes[0][1], bins = 10, color = "mediumslateblue");
axes[0][1].set(xlabel = 'Distribution of Volume Sold (Gallons)');
draw_axvlines(pp, "Volume Sold (Gallons)");



sns.boxplot(dfx["Volume Sold (Liters)"], ax = axes[1][0], color = "mediumslateblue");
axes[1][0].set(xlabel = 'Distribution of Volume Sold (Liters)');

pp = sns.distplot(dfx["Volume Sold (Liters)"], ax = axes[1][1], bins = 10, color = "mediumslateblue");
axes[1][1].set(xlabel = 'Distribution of Volume Sold (Liters)');
draw_axvlines(pp, "Volume Sold (Liters)")



sns.boxplot(dfx["Pack"], ax = axes[2][0], color = "mediumslateblue");
axes[2][0].set(xlabel = 'Distribution of Pack');

pp = sns.distplot(dfx["Pack"], ax = axes[2][1], color = "mediumslateblue");
axes[2][1].set(xlabel = 'Distribution of Pack');
draw_axvlines(pp, "Pack")

In [ ]:
def groupby_get_bottle_count(tdf, col):
    tdf = tdf.groupby([col, "Bottles Sold"])["Bottles Sold"].count().reset_index(level = 0)
    tdf.columns = [col, "count"]
    tdf = tdf.reset_index()
    return tdf

In [ ]:
df['Sale (Dollars)'] = df['Sale (Dollars)'].str.replace('$','').astype(float)




In [ ]:
total_sale_store = df.groupby('Store Name').agg({'Sale (Dollars)':'sum'}).reset_index().sort_values('Sale (Dollars)',ascending =False)
tts = total_sale_store.head(50)



In [ ]:
a1 = (15, 15)
fig, ax = plt.subplots(figsize=a1)
plot= sns.barplot(x="Store Name", y="Sale (Dollars)", data=tts,palette ="deep")
plot.set_xticklabels(ax.get_xticklabels(),rotation=90)
plot.set_title('Best Store Sale ')
ax.set_ylabel('Sale (Dollars)')
ax.set_xlabel('Store Name')


In [ ]:
total_sale_bottle = df.groupby('Item Description').agg({'Sale (Dollars)':'sum'}).reset_index().sort_values('Sale (Dollars)',ascending =False)
ttb = total_sale_bottle.head(50)
ttb

In [ ]:
a1 = (15, 15)
fig, ax = plt.subplots(figsize=a1)
plot= sns.barplot(x="Item Description", y="Sale (Dollars)", data=ttb,palette ="deep")
plot.set_xticklabels(ax.get_xticklabels(),rotation=90)
plot.set_title('Best Liquor Sale ')
ax.set_ylabel('Sale (Dollars)')
ax.set_xlabel('Item Description')


In [ ]:
total_sale_cat = df.groupby('Category Name').agg({'Sale (Dollars)':'sum'}).reset_index().sort_values('Sale (Dollars)',ascending =False)
ttc = total_sale_cat.head(50)


a1 = (15, 15)
fig, ax = plt.subplots(figsize=a1)
plot= sns.barplot(x="Category Name", y="Sale (Dollars)", data=ttc,palette ="deep")
plot.set_xticklabels(ax.get_xticklabels(),rotation=90)
plot.set_title('Best Liquor Sale ')
ax.set_ylabel('Sale (Dollars)')
ax.set_xlabel('Category')



In [ ]:
total_sale_cat1 = df.groupby('Category Name').agg({'Sale (Dollars)':'sum','Volume Sold (Liters)':'sum'}).reset_index().sort_values('Sale (Dollars)',ascending =False)
ttc1 = total_sale_cat1.head(50)
ttc1['price/lt'] = ttc1['Sale (Dollars)']/ttc1['Volume Sold (Liters)']
print(ttc1)
print(ttc1.describe())

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [ ]:
month_sales_canWhis = df[df['Category Name'] == 'CANADIAN WHISKIES'].groupby('Month', as_index=False)['Sale (Dollars)'].sum()
print(month_sales_canWhis)

x = month_sales_canWhis['Month']
y1 = month_sales_canWhis['Sale (Dollars)']

ax = sns.pointplot(x,y1,data=month_sales_canWhis,color="black",markers="H")
plt.axhline(y1.mean(),color="k",linestyle="dotted",linewidth=2)

ax.set_title('CANADIAN WHISKIES monthly sales') 
  

In [ ]:
month_sales_VODKA80PROOF = df[df['Category Name'] == 'VODKA 80 PROOF'].groupby('Month', as_index=False)['Sale (Dollars)'].sum()

print(month_sales_VODKA80PROOF)

x = month_sales_VODKA80PROOF['Month']
y2 = month_sales_VODKA80PROOF['Sale (Dollars)']

ax = sns.pointplot(x,y2,data=month_sales_VODKA80PROOF,color="black",markers="H")
plt.axhline(y1.mean(),color="k",linestyle="dotted",linewidth=2)

ax.set_title('VODKA80PROOF monthly sales') 




In [ ]:
month_sales_SPICEDRUM = df[df['Category Name'] == 'SPICED RUM'].groupby('Month', as_index=False)['Sale (Dollars)'].sum()
print(month_sales_SPICEDRUM)

x = month_sales_SPICEDRUM['Month']
y3 = month_sales_SPICEDRUM['Sale (Dollars)']

ax = sns.pointplot(x,y3,data=month_sales_SPICEDRUM,color="black",markers="H")
plt.axhline(y3.mean(),color="k",linestyle="dotted",linewidth=2)

ax.set_title('SPICEDRUM monthly sales') 




In [ ]:
data = [y1,y2,y3]
headers = ["CANADIAN WHISKIES","VODKA80PROOF","SPICEDRUM"]
top3_sell = pd.concat(data, axis=1, keys=headers)
top3_sell.index = top3_sell.index +1

ax = top3_sell.plot(linewidth=2, fontsize=8);
ax.set_xlabel('Month');
ax.legend(fontsize=8);



Dapat dilihat bahwa dari penjualan top 3 minuman memiliki kesamaan yaitu:
1. Terjadi peningkatan penjualan signifikan secara rata-rata setiap tahun dari data yang ada pada bulan Januari hingga Juni, serta bulan Oktober dan Desember.
2. Pada bulan sisanya, terjadi penurunan secara penjualan dari bulan pendahulunya.
3. Dari hasil tersebut teridentifikasi terdapat masalah yang tidak diketahui dari penurunan pada bulan yang dimaksud.
4. Top 3 minuman yang paling laku dijual memiliki kisaran harga per liter yang tidak lebih dari usd 15,2 untuk data semua tahun.